In [1]:
import os
# set current directory as the repo base root
os.chdir('\\'.join(os.getcwd().split('\\')[:-1]))

In [2]:
from zeroCPR import engine
import pandas as pd

# initiate engine
myagent = engine.agent(groq_api_key='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')

c:\Users\michelangiolo.m\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# https://www.kaggle.com/datasets/ramzanzdemir/online-retail-gift-products
df = pd.read_excel('notebooks/df_raw.xlsx')
product_list = list(set(df['Description'].dropna().tolist()))
product_list = [x for x in product_list if isinstance(x, str)]
product_list = [x.strip() for x in product_list]
product_list

# encode products (should only be runned once)
df_encoded = myagent.encode_products(product_list)
df_encoded.to_parquet('notebooks/df_encoded.parquet', index=None)

100%|██████████| 4678/4678 [00:57<00:00, 80.82it/s]


In [3]:
# if you have already runned the encoding module
# you can upload encoded products from parquet to save time
df_encoded = pd.read_parquet('notebooks/df_encoded.parquet')
myagent.upload_encoded_products(df_encoded)
product_list = df_encoded.raw.tolist()

### find complementaries of a single product

In [4]:
df_candidates, df_filtered = myagent.find_product_complementaries(['pizza'])
display(df_filtered)

,index,product_name,llm_product,recommended_product,reasoning,score
0,3774,pizza,soda,CHOCOLATE HOT WATER BOTTLE,There is no clear relation between pizza and a...,0
1,1208,pizza,fork,CAST IRON HOOK GARDEN FORK,There is no clear relation between pizza and a...,0
2,3538,pizza,knife,MEDIUM CHINESE STYLE SCISSOR,There is no clear relation between pizza and s...,0
3,2461,pizza,plate,RED SPOTTY PLATE,"A plate can be used to serve pizza, making thi...",1
4,3784,pizza,salad,FRUIT SALAD BAG CHARM,There is no clear relation between pizza and a...,0
5,2581,pizza,ketchup,SANDWICH BATH SPONGE,There is no clear relation between pizza and a...,0
6,1940,pizza,mustard,Rusty,"Rusty is not a clear product name, but it seem...",0
7,3781,pizza,hot sauce,CHICK GREY HOT WATER BOTTLE,There is no clear relation between pizza and a...,0
8,3859,pizza,ice cream,CHOCOLATE ICE CREAM CANDLE,There is no clear relation between pizza and a...,0
9,2478,pizza,cookies,GINGERBREAD MAN COOKIE CUTTER,There is no clear relation between pizza and a...,0


In [5]:
#we can explore the comparison between complementary and similar products
print('complementary\t', df_filtered[df_filtered['score']==1]['recommended_product'].tolist())
print('similar\t\t', myagent.search_similar('pizza', k=len(df_filtered[df_filtered['score']==1])))

complementary	 ['RED SPOTTY PLATE', 'SET/20 COLLAGE PAPER NAPKIN', 'ORANGE TV TRAY TABLE', 'RED SPOTTY TABLECLOTH']
similar		 ['PIZZA SLICE DISH', 'PIZZA DISH SLICE', 'PARTY PIZZA DISH BLUE POLKADOT', 'PARTY PIZZA DISH PINK POLKADOT']


### find the complementary of 10 products

In [6]:
df_complementaries = myagent.find_product_complementaries(product_list[50:60], max_retries=4)
df_complementaries.to_parquet('notebooks/df_complementaries.parquet', index=None)

** CREAM DELPHINIUM ARTIFICIAL FLOWER
** PINK PAISLEY PHOTO ALBUM
** stock take
** WHITE CHOCOLATE SCENT CANDLE
** WHITE VINT ART DECO CRYSTAL NECKLAC
** PINK CRYSTAL HEART PHONE CHARM
** RED 3 PIECE MINI DOTS CUTLERY SET
** ASS COL SMALL SAND GECKO P'WEIGHT
** SET OF 3 CONEY ISLAND OVAL BOXES
** SUNJAR LED NIGHT NIGHT LIGHT


In [8]:
# let us check the list of valid complementaries
df_complementaries[df_complementaries['score']==1]

,index,product_name,llm_product,recommended_product,reasoning,score
0,1484,CREAM DELPHINIUM ARTIFICIAL FLOWER,FLORAL WIRE,ASSORTED FLORAL SECATEURS,Floral scissors to trim and shape the artifici...,1
1,3891,CREAM DELPHINIUM ARTIFICIAL FLOWER,FLOWER VASE,ORANGE FELT VASE + FLOWERS,Vase to complement the artificial flower,1
6,1502,CREAM DELPHINIUM ARTIFICIAL FLOWER,CRAFTING SUPPLIES,BASKET OF FLOWERS SEWING KIT,Floral-themed sewing kit to complement the art...,1
8,1947,CREAM DELPHINIUM ARTIFICIAL FLOWER,BOUQUET HOLDER,FLOWER FAIRY INCENSE BOUQUET,Incense bouquet with a floral theme,1
1,2835,PINK PAISLEY PHOTO ALBUM,PHOTO LABELS OR CATEGORY TABS,CANDY BUTTON PHOTO ALBUM,"similar product, similar category, variation i...",1
...,...,...,...,...,...,...
12,4176,SUNJAR LED NIGHT NIGHT LIGHT,Corner Shelf,BLUE METAL WALL SHELF,Can be used together to create a cozy ambiance,1
14,2420,SUNJAR LED NIGHT NIGHT LIGHT,String Lights,STRING OF 20 ROSE LIGHTS,Complementary product to enhance ambiance,1
15,1533,SUNJAR LED NIGHT NIGHT LIGHT,End Table,PINK KASHMIRI OCCASIONAL TABLE,Can be paired together to create a soothing at...,1
16,3126,SUNJAR LED NIGHT NIGHT LIGHT,Display Case Lighting,ASSORTED COLOUR T-LIGHT HOLDER,Complementary product to enhance ambiance,1
